<a href="https://colab.research.google.com/github/jekim526/IE-541-Project/blob/Parsing_Test/Copy_of_BasicGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize
Load problem data, install packages

In [2]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
!pip install deap
!git clone https://github.com/jekim526/IE-541-Project.git
!git --git-dir=/content/IE-541-Project/.git pull
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
sys.path.append('/content/IE-541-Project')
import knapsack_EA_functions as ea

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'IE-541-Project' already exists and is not an empty directory.
Already up to date.


In [3]:
url = 'https://raw.githubusercontent.com/jekim526/IE-541-Project/main/data/Z_r_100_25_1.csv' # Get the "Raw" link
df1_o = pd.read_csv(url)
df1 = pd.DataFrame(df1_o).to_numpy() # DataFrame to nummpy array
n_size = len(df1) - 1

# Array of item value
item_value = [0] * n_size
for x in range(0,n_size):
  item_value[x] = df1[1,x+1]

# Array of item weight
item_weight = [0] * n_size
for x in range(0,n_size):
  item_weight[x] = df1[0,x+1]

# n x n array of joint profit
joint_profit = np.zeros((n_size,n_size))
for x in range(0, n_size-1):
  for y in range(0, n_size):
    joint_profit[x,y] = df1[x+2,y+1]

item_weight = np.array(item_weight)
item_value = np.array(item_value)

## Prepare the GA input and solve the instance via GA

In [4]:
# -------  NOTES: ------------------------------------------------------------------------------
# instance_setings should be a tuple contains:
# {item_value, item_weight, joint_profit, capacities}
#                 capacities = {capacity1, capacity2, ...}
#
# evolution_general_parameters should be a tuple contains:
# {popsize, swap_prob, mute_prob, penalty}, in which:
#    swap_prob is independent probability for swap at each point in uniform crossover.
#    mute_prob is independent probability for each attribute to be flipped in flip-bit mutation.
#    penalty is exit of the weight. 
#
# evolution_specify_parameters should be a tuple contains: {CXPB, MUTPB},In which:
#    CXPB is the probability with which two individuals
#          are crossed
#    MUTPB is the probability for mutating an individual
#    MAX_GEN is the maximum generation threshold
#    STOP_GEN is the threshold of no progress generations

''' the maximize covalue_case '''
max_c = 0; max_c = max(max_c,max(np.sum(joint_profit,axis = 1))); max_c = max(max_c,max(np.sum(joint_profit,axis = 0)))
max_i = max(item_value)

''' prepare this solver inputs: '''
#initialize evolution parameter setting
evolution_general_parameters = (100, 0.1, 0.1, max_i + max_c)
evolution_specify_parameters = (0.5, 0.25, 1000, 100)
num_of_knapsack = 3
capacity = (sum(item_weight)/num_of_knapsack)*0.8 # 80% of the sum of all item weights divided by the number of knapsacks
capacities = (capacity,)*num_of_knapsack
#initialize instance setting
instance_settings = (item_value, item_weight, joint_profit, capacities)

In [12]:
''' solve via GA in one line! ''' 
#objf_type: 
# 1: the first objective function (maximum the total profit)
# 2: maximum the negative of total_weight
# 3: maximum the profit of the package with minmum of profit

#best_ind,pop,num_gen = ea.perform_GA_base(1, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=False)

best_ind1,pop,num_gen = ea.perform_GA_base(1, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=False)
best_ind2,pop,num_gen = ea.perform_GA_base(2, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=False)
best_ind3,pop,num_gen = ea.perform_GA_base(3, instance_settings, evolution_general_parameters, evolution_specify_parameters, PRINT=False)
best_ind1.fitness.values
best_ind2.fitness.values
best_ind3.fitness.values
print(best_ind1.fitness.values)
print(best_ind2.fitness.values)
print(best_ind3.fitness.values)

/usr/local/lib/python3.8/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.8/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


(22668.0,)
(0.0,)
(6645.0,)


In [11]:
#Output the best individual as .csv file
import pandas as pd

# Find out which knapsack is each individual going
ind_knap_location1 = [0] * len(best_ind1)
ind_knap_location2 = [0] * len(best_ind2)
ind_knap_location3 = [0] * len(best_ind3)

for x in range(0, len(best_ind1)):
  i = 1
  if i in best_ind1[x]:
    ind_knap_location1[x] = tuple(best_ind1[x]).index(1) + 1
  else:
    ind_knap_location1[x] = 0

for x in range(0, len(best_ind2)):
  i = 1
  if i in best_ind2[x]:
    ind_knap_location2[x] = tuple(best_ind2[x]).index(1) + 1
  else:
    ind_knap_location2[x] = 0

for x in range(0, len(best_ind3)):
  i = 1
  if i in best_ind3[x]:
    ind_knap_location3[x] = tuple(best_ind3[x]).index(1) + 1
  else:
    ind_knap_location3[x] = 0

locations ={
    tuple(ind_knap_location1),
    tuple(ind_knap_location2),
    tuple(ind_knap_location3)
}

df_location = pd.DataFrame(locations)
instance_name = "Z_r_100_25_1"
data_function = [(instance_name, 1 ,num_of_knapsack),(instance_name, 2, num_of_knapsack),(instance_name, 3, num_of_knapsack)]

df_info = pd.DataFrame(data_function)
df_info = df_info.T

df_for_analysis = df_info.append(df_location.T, ignore_index = True)
df_for_analysis.to_csv("/content/for_analysis_"+instance_name+".csv")
